### 설문 조사 데이터를 오버샘플링 하기 위한 코드
- 샘플링 수를 늘려보자 !

#### 필요한 라이브러리 로드

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
# 랜덤 오버 샘플링을 위한 라이브러리
from imblearn.over_sampling import RandomOverSampler

#### 파일 위치 지정

In [3]:
file_path = './refer/output/'

#### 설문 데이터 전처리

In [4]:
def survey_processing(file_path = f'{file_path}survey.csv', output_path = f'{file_path}survey_preprocessed.csv'):
    # CSV 파일을 불러옵니다.
    survey_df = pd.read_csv(file_path)
    
    # 우선순위에 대한 가중치 매핑을 정의합니다.
    priority_weights = {
        "건강": "health",
        "여가시간": "free_time",
        "학업 및 자기계발": "edu",
        "업무": "work",
        "집안일": "chores",
        None: "category_else"
    }
    
    # 각 우선순위 컬럼에 대해 가중치를 적용합니다.
    for i in range(1, 6):
        column_name = f"본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [{i} 순위]"
        survey_df[column_name] = survey_df[column_name].map(priority_weights)
    
    # 필요한 컬럼들만 선택하고 리네임합니다.
    columns_to_keep = [
        "설문자의 나이(만)는 어떻게 되십니까?",
        "설문자의 해당사항을 체크해주세요.",
        "설문자의 MBTI는 무엇입니까?",
        "설문자의 성별은 어떻게 되십니까?",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [4 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [5 순위]"
    ]
    
    survey_df = survey_df[columns_to_keep]
    
    # 컬럼명을 변환합니다.
    column_mapping = {
        "설문자의 나이(만)는 어떻게 되십니까?": "age",
        "설문자의 해당사항을 체크해주세요.": "job",
        "설문자의 MBTI는 무엇입니까?": "mbti",
        "설문자의 성별은 어떻게 되십니까?": "gender",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위]": "priority_1",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위]": "priority_2",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위]": "priority_3",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [4 순위]": "priority_4",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [5 순위]": "priority_5"
    }
    
    survey_df.rename(columns=column_mapping, inplace=True)
    
    # 새로운 컬럼 'category_else'를 추가하고 모든 값을 0으로 설정합니다.
    survey_df["category_else"] = 0
    
    # 나이, 직업, MBTI 컬럼을 숫자 코드로 변환합니다.
    age_mapping = {
        "0~14세": (0, 14),
        "15~19세": (15, 19),
        "20~24세": (20, 24),
        "25~30세": (25, 30),
        "31세 이상": (31, 45)
    }
    job_mapping = {
        "초/중학생": "000",
        "고등학생": "001",
        "대학생 / 저학년 (1-2학년)": "002",
        "대학생 / 고학년(3-4학년)": "003",
        "구직자": "004",
        "직장인": "005",
        "자영업자": "006",
        "프리랜서": "007",
        "주부": "008",
        "기타": "009"
    }
    mbti_mapping = {
        "INTJ": "00",
        "INTP": "01",
        "ENTJ": "02",
        "ENTP": "03",
        "INFJ": "04",
        "INFP": "05",
        "ENFJ": "06",
        "ENFP": "07",
        "ISTJ": "08",
        "ISFJ": "09",
        "ESTJ": "10",
        "ESFJ": "11",
        "ISTP": "12",
        "ISFP": "13",
        "ESTP": "14",
        "ESFP": "15"
    }
    gender_mapping = {
        "남": 0,
        "여": 1
    }

    # 나이, 직업, MBTI, 성별 컬럼을 숫자 코드로 변환합니다.
    survey_df['age'] = survey_df['age'].map(lambda x: np.random.randint(age_mapping[x][0], age_mapping[x][1] + 1))
    survey_df['job'] = survey_df['job'].map(job_mapping)
    survey_df['mbti'] = survey_df['mbti'].str.upper().map(mbti_mapping)
    survey_df['gender'] = survey_df['gender'].map(gender_mapping)

    # 우선순위별로 각 항목에 가중치를 적용한 값을 할당합니다.
    work_col = []
    edu_col = []
    free_time_col = []
    health_col = []
    chores_col = []

    for _, row in survey_df.iterrows():
        work = 0
        edu = 0
        free_time = 0
        health = 0
        chores = 0

        for i in range(1, 6):
            if row[f'priority_{i}'] == 'work':
                work = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'edu':
                edu = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'free_time':
                free_time = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'health':
                health = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'chores':
                chores = 1 - (i - 1) * 0.25
        
        work_col.append(work)
        edu_col.append(edu)
        free_time_col.append(free_time)
        health_col.append(health)
        chores_col.append(chores)
    
    survey_df['work'] = work_col
    survey_df['edu'] = edu_col
    survey_df['free_time'] = free_time_col
    survey_df['health'] = health_col
    survey_df['chores'] = chores_col
    
    # 사용하지 않는 우선순위 컬럼 삭제
    survey_df.drop(columns=['priority_1', 'priority_2', 'priority_3', 'priority_4', 'priority_5'], inplace=True)
    
    # 컬럼 순서 재정렬
    survey_df = survey_df[['age', 'job', 'mbti', 'gender', 'work', 'edu', 'free_time', 'health', 'chores', 'category_else']]
    
    # 수정된 데이터프레임을 새로운 CSV 파일로 저장합니다.
    survey_df.to_csv(output_path, index=False)

In [5]:
survey_processing()  # 함수를 호출하여 변환을 수행합니다.

#### 원본 데이터 단순 복제로 데이터 수를 50만개로 늘리기

In [6]:
# 데이터 로드
data = pd.read_csv(f'{file_path}survey_preprocessed.csv')

# 원하는 샘플 수
target_samples = 500000
replication_factor = target_samples // len(data)  # 필요한 복제 횟수 계산
additional_samples = target_samples % len(data)   # 추가로 필요한 샘플 수

# 데이터 복제
oversampled_data = pd.concat([data] * replication_factor + [data.iloc[:additional_samples]])

# 결과 저장
oversampled_data.to_csv(f'{file_path}survey_replicated.csv', index=False)

print(f"데이터 복제 완료: 총 샘플 수 {len(oversampled_data)}")


데이터 복제 완료: 총 샘플 수 500000


#### 랜덤 오버 샘플링을 통해 전체 데이터의 수를 늘리기

- 단, 랜덤 오버 샘플링은 이산형 데이터에 사용이 가능
- 따라서 '0', '0.25', '0.5', '0.75', '1'을 범주화하여 제한을 우회
- 이후 다시 범주형 데이터를 수치로 롤백하여 원본 데이터의 형식을 유지
- 이 외의 숫자가 혹시 포함되어 있다면 디버깅을 위해 -1로 설정

- 랜덤 오버 샘플링을 위해 각 열을 독립적으로 오버 샘플링 후 다시 합쳐주는 방식 선택

- 하지만 지금까지는 실패~
    - 각 열을 독립적으로 오버 샘플링 하다보니 각 열의 인덱스를 합쳐야 NA가 없음
    - 그러나 랜덤으로 샘플링 하기 때문에 각 파일별 인덱스 수가 달라짐
    - 따라서 망함 :,<

In [7]:
# 메모리 사용을 줄이기 위해 범주에 대해 개별 오버 샘플링 후 저장
'''
def discretize(value):
    if value == 0:
        return 'Very Low'
    elif value == 0.25:
        return 'Low'
    elif value == 0.5:
        return 'Medium'
    elif value == 0.75:
        return 'High'
    elif value == 1:
        return 'Very High'
    return 'Unknown'

def continuousize(category):
    mapping = {'Very Low': 0, 'Low': 0.25, 'Medium': 0.5, 'High': 0.75, 'Very High': 1}
    return mapping.get(category, -1)

# 데이터 로드
original_data = pd.read_csv(f'{file_path}survey_replicated.csv')
categories = ['work', 'edu', 'free_time', 'health', 'chores']

# 범주화 및 원-핫 인코딩
X_encoded = pd.get_dummies(original_data.drop(categories, axis=1))
ros = RandomOverSampler(random_state=42)

# 각 범주에 대해 오버샘플링 적용 및 저장
for category in categories:
    y = original_data[category].apply(discretize)
    X_resampled, y_resampled = ros.fit_resample(X_encoded, y)
    y_resampled = y_resampled.apply(continuousize)
    df_resampled = pd.DataFrame(X_resampled, columns=X_encoded.columns)
    df_resampled[category] = y_resampled
    df_resampled.to_csv(f'{file_path}survey_random_oversampled_{category}.csv', index=False)
    print(f"오버샘플링 데이터 저장 완료: survey_random_oversampled_{category}.csv, 샘플 수: {len(df_resampled)}")

print("모든 범주에 대한 오버샘플링 데이터 저장 완료")
'''

'\ndef discretize(value):\n    if value == 0:\n        return \'Very Low\'\n    elif value == 0.25:\n        return \'Low\'\n    elif value == 0.5:\n        return \'Medium\'\n    elif value == 0.75:\n        return \'High\'\n    elif value == 1:\n        return \'Very High\'\n    return \'Unknown\'\n\ndef continuousize(category):\n    mapping = {\'Very Low\': 0, \'Low\': 0.25, \'Medium\': 0.5, \'High\': 0.75, \'Very High\': 1}\n    return mapping.get(category, -1)\n\n# 데이터 로드\noriginal_data = pd.read_csv(f\'{file_path}survey_replicated.csv\')\ncategories = [\'work\', \'edu\', \'free_time\', \'health\', \'chores\']\n\n# 범주화 및 원-핫 인코딩\nX_encoded = pd.get_dummies(original_data.drop(categories, axis=1))\nros = RandomOverSampler(random_state=42)\n\n# 각 범주에 대해 오버샘플링 적용 및 저장\nfor category in categories:\n    y = original_data[category].apply(discretize)\n    X_resampled, y_resampled = ros.fit_resample(X_encoded, y)\n    y_resampled = y_resampled.apply(continuousize)\n    df_resampled = pd.Da

In [8]:
# 모든 열에 처리시 메모리 사용량이 지나치게 높기에 위 코드를 통해 각 범주별 개별 오버샘플링 적용 후 합치는 방식 선택
'''
# 데이터 로드 및 함수 정의
def discretize(value):
    if value == 0:
        return 'Very Low'
    elif value == 0.25:
        return 'Low'
    elif value == 0.5:
        return 'Medium'
    elif value == 0.75:
        return 'High'
    elif value == 1:
        return 'Very High'
    return 'Unknown'

def continuousize(category):
    mapping = {'Very Low': 0, 'Low': 0.25, 'Medium': 0.5, 'High': 0.75, 'Very High': 1}
    return mapping.get(category, -1)

original_data = pd.read_csv(f'{file_path}survey_replicated.csv')
categories = ['work', 'edu', 'free_time', 'health', 'chores']

# 범주화 및 원-핫 인코딩
X_encoded = pd.get_dummies(original_data.drop(categories, axis=1))
oversampled_dataframes = []
ros = RandomOverSampler(random_state=42)

# 각 범주에 대해 오버샘플링 적용
for category in categories:
    y = original_data[category].apply(discretize)
    X_resampled, y_resampled = ros.fit_resample(X_encoded, y)
    y_resampled = y_resampled.apply(continuousize)
    df_resampled = pd.DataFrame(X_resampled, columns=X_encoded.columns)
    df_resampled[category] = y_resampled
    oversampled_dataframes.append(df_resampled)

# 모든 결과를 하나의 DataFrame으로 병합
final_df = oversampled_dataframes[0]
for df in oversampled_dataframes[1:]:
    final_df = final_df.merge(df, on=list(X_encoded.columns), how='inner')

# 반복적 오버샘플링으로 추가 증가
for _ in range(200):  # 반복 횟수에 따라 조정 가능
    final_df, _ = ros.fit_resample(final_df, final_df.columns)

# 결과 저장
final_df.to_csv(f'{file_path}survey_random_oversampled.csv', index=False)
print(f"오버샘플링 데이터 저장 완료: survey_random_oversampled.csv, 샘플 수: {len(final_df)}")
'''

'\n# 데이터 로드 및 함수 정의\ndef discretize(value):\n    if value == 0:\n        return \'Very Low\'\n    elif value == 0.25:\n        return \'Low\'\n    elif value == 0.5:\n        return \'Medium\'\n    elif value == 0.75:\n        return \'High\'\n    elif value == 1:\n        return \'Very High\'\n    return \'Unknown\'\n\ndef continuousize(category):\n    mapping = {\'Very Low\': 0, \'Low\': 0.25, \'Medium\': 0.5, \'High\': 0.75, \'Very High\': 1}\n    return mapping.get(category, -1)\n\noriginal_data = pd.read_csv(f\'{file_path}survey_replicated.csv\')\ncategories = [\'work\', \'edu\', \'free_time\', \'health\', \'chores\']\n\n# 범주화 및 원-핫 인코딩\nX_encoded = pd.get_dummies(original_data.drop(categories, axis=1))\noversampled_dataframes = []\nros = RandomOverSampler(random_state=42)\n\n# 각 범주에 대해 오버샘플링 적용\nfor category in categories:\n    y = original_data[category].apply(discretize)\n    X_resampled, y_resampled = ros.fit_resample(X_encoded, y)\n    y_resampled = y_resampled.apply(continuo

In [9]:
'''
import glob
import os

# 모든 CSV 파일의 경로를 리스트로 로드
csv_files = glob.glob(os.path.join(file_path, 'survey_random*.csv'))

# CSV 파일 경로 출력 (디버깅용)
print("CSV files found:", csv_files)

# CSV 파일이 존재하는지 확인
if not csv_files:
    raise ValueError("No CSV files found in the specified directory.")

# 모든 CSV 파일을 읽어 하나의 데이터프레임으로 병합
dataframes = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        dataframes.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

# 데이터프레임 병합
if dataframes:
    merged_dataframe = pd.concat(dataframes, ignore_index=True)
    # 합쳐진 데이터프레임을 새로운 CSV 파일로 저장
    merged_dataframe.to_csv(f"{file_path}merged_survey_data.csv", index=False)
else:
    raise ValueError("No valid CSV files to concatenate.")
'''

'\nimport glob\nimport os\n\n# 모든 CSV 파일의 경로를 리스트로 로드\ncsv_files = glob.glob(os.path.join(file_path, \'survey_random*.csv\'))\n\n# CSV 파일 경로 출력 (디버깅용)\nprint("CSV files found:", csv_files)\n\n# CSV 파일이 존재하는지 확인\nif not csv_files:\n    raise ValueError("No CSV files found in the specified directory.")\n\n# 모든 CSV 파일을 읽어 하나의 데이터프레임으로 병합\ndataframes = []\nfor file in csv_files:\n    try:\n        df = pd.read_csv(file)\n        dataframes.append(df)\n    except Exception as e:\n        print(f"Error reading {file}: {e}")\n\n# 데이터프레임 병합\nif dataframes:\n    merged_dataframe = pd.concat(dataframes, ignore_index=True)\n    # 합쳐진 데이터프레임을 새로운 CSV 파일로 저장\n    merged_dataframe.to_csv(f"{file_path}merged_survey_data.csv", index=False)\nelse:\n    raise ValueError("No valid CSV files to concatenate.")\n'

#### 단순 오버 샘플링을 통해 늘어난 데이터 수를 바탕으로 smote 적용

In [10]:
# 데이터 로드
original_data = pd.read_csv(f'{file_path}survey_replicated.csv')

# 데이터 스케일링 및 클러스터링
scaler = StandardScaler()
y_scaled = scaler.fit_transform(original_data[['work', 'edu', 'free_time', 'health', 'chores']])

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(y_scaled)


# SMOTE 적용 (연속형 데이터 포함)
# n_neighbors 값 조정: 가장 작은 클러스터의 크기보다 작게 설정
min_cluster_size = min(pd.Series(clusters).value_counts())
n_neighbors = max(min(2, min_cluster_size - 1), 1)  # 최소 1, 최대 (가장 작은 클러스터 크기 - 1) 사이
smote = SMOTE(random_state=42, k_neighbors=n_neighbors)
X_resampled, clusters_resampled = smote.fit_resample(original_data, clusters)

# 연속형 데이터 복원: 각 클러스터의 중심을 사용하여 연속형 데이터를 복원
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
oversampled_y = [cluster_centers[cluster] for cluster in clusters_resampled]

# 오버샘플링된 데이터 프레임 생성
oversampled_df = pd.DataFrame(X_resampled, columns=original_data.columns)
oversampled_df[['work', 'edu', 'free_time', 'health', 'chores']] = pd.DataFrame(oversampled_y, columns=['work', 'edu', 'free_time', 'health', 'chores'])

# 결과 저장 및 출력
oversampled_df.to_csv(f'{file_path}survey_oversampled.csv', index=False)
print("오버샘플링 데이터 저장 완료: survey_oversampled.csv")


#### 오버 샘플링 된 데이터들을 정제하는 과정

In [ ]:
'''
# 오버샘플링된 데이터 로드
oversampled_data = pd.read_csv(f'{file_path}survey_oversampled.csv')

# 'job' 관련 열 추출 및 변환
job_columns = [col for col in oversampled_data.columns if col.startswith('job_')]
oversampled_data['job'] = oversampled_data[job_columns].idxmax(axis=1).apply(lambda x: x.split('_')[1] if pd.notna(x) else None)

# 'mbti' 관련 열 추출 및 변환
mbti_columns = [col for col in oversampled_data.columns if col.startswith('mbti_')]
oversampled_data['mbti'] = oversampled_data[mbti_columns].idxmax(axis=1).apply(lambda x: x.split('_')[1] if pd.notna(x) else None)

# 불필요한 원-핫 인코딩 열 제거
oversampled_data.drop(columns=job_columns + mbti_columns, inplace=True)

# 결과 저장
oversampled_data.to_csv(f'{file_path}survey_oversampled_fixed.csv', index=False)

print("데이터 업데이트 완료: survey_oversampled_fixed.csv")
'''

'\n# 오버샘플링된 데이터 로드\noversampled_data = pd.read_csv(f\'{file_path}survey_oversampled.csv\')\n\n# \'job\' 관련 열 추출 및 변환\njob_columns = [col for col in oversampled_data.columns if col.startswith(\'job_\')]\noversampled_data[\'job\'] = oversampled_data[job_columns].idxmax(axis=1).apply(lambda x: x.split(\'_\')[1] if pd.notna(x) else None)\n\n# \'mbti\' 관련 열 추출 및 변환\nmbti_columns = [col for col in oversampled_data.columns if col.startswith(\'mbti_\')]\noversampled_data[\'mbti\'] = oversampled_data[mbti_columns].idxmax(axis=1).apply(lambda x: x.split(\'_\')[1] if pd.notna(x) else None)\n\n# 불필요한 원-핫 인코딩 열 제거\noversampled_data.drop(columns=job_columns + mbti_columns, inplace=True)\n\n# 결과 저장\noversampled_data.to_csv(f\'{file_path}survey_oversampled_fixed.csv\', index=False)\n\nprint("데이터 업데이트 완료: survey_oversampled_fixed.csv")\n'

In [ ]:
'''
# 마지막으로 value 수정
fix_data = pd.read_csv(f'{file_path}survey_oversampled.csv')

# 열 순서 재배열
columns_ordered = ['age', 'gender', 'job', 'mbti', 'work', 'edu', 'free_time', 'health', 'chores']
fix_data = fix_data[columns_ordered]

fix_data.to_csv(f'{file_path}survey_data.csv')
'''

NameError: name 'pd' is not defined

In [ ]:
fix_data.describe()

,age,gender,job,mbti,work,edu,free_time,health,chores
count,600003.000000,600003.000000,600003.000000,600003.000000,600003.000000,600003.000000,600003.000000,600003.000000,600003.000000
mean,28.799424,0.589380,4.347768,7.354457,0.599243,0.422304,0.496634,0.684763,0.196383
std,7.351794,0.491947,1.546530,4.525648,0.258853,0.157070,0.243146,0.262233,0.182342
min,12.000000,0.000000,0.000000,0.000000,0.233333,0.263885,0.152780,0.318182,0.011364
25%,25.000000,0.000000,3.000000,4.000000,0.233333,0.263885,0.152780,0.318182,0.011364
50%,27.000000,1.000000,4.000000,7.000000,0.772729,0.366667,0.666667,0.819440,0.133333
75%,32.000000,1.000000,5.000000,12.000000,0.791668,0.636362,0.670454,0.916667,0.444452
max,45.000000,1.000000,8.000000,15.000000,0.791668,0.636362,0.670454,0.916667,0.444452


----------------------------------------------------

### 샘플링 이후 스케일링 방법으로 데이터 생성

- 랜덤 선택 후 증식 방법으로 설문 조사 분포를 유지하며 데이터 증식
- 랜덤 샘플링 이후 smote 적용하여 데이터 불균형 해소
- 이후 데이터 전처리

In [23]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# CSV 파일 경로
file_path = './refer/output/'
data_path = f'{file_path}survey.csv'
output_path = f'{file_path}survey_oversampled_v2.csv'
preprocessed_output_path = f'{file_path}survey_preprocessed_v2.csv'

# CSV 파일 불러오기
survey_df = pd.read_csv(data_path)

# 필요한 컬럼만 선택
columns_to_keep = [
    "설문자의 나이(만)는 어떻게 되십니까?",
    "설문자의 해당사항을 체크해주세요.", # 직업
    "설문자의 MBTI는 무엇입니까?",
    "설문자의 성별은 어떻게 되십니까?",
    "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위]",
    "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위]",
    "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위]",
    "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [4 순위]",
    "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [5 순위]"
]
survey_df = survey_df[columns_to_keep]

# 오버샘플링을 수행하는 함수
def oversample_data(df, target_size=700000):
    current_size = len(df)
    if (target_size - current_size) > current_size:
        oversample_count = target_size - current_size
        oversampled_df = resample(df, replace=True, n_samples=oversample_count, random_state=42)
        return pd.concat([df, oversampled_df], axis=0)
    else:
        return resample(df, replace=True, n_samples=target_size, random_state=42)

# 데이터를 70만 개로 오버샘플링
oversampled_survey_df = oversample_data(survey_df, target_size=700000)

# 더미 변수 생성
categorical_features = columns_to_keep

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# 전처리 파이프라인
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# 데이터 변환
X = pipeline.fit_transform(oversampled_survey_df)

# KMeans 클러스터링
# 5순위 특성값에 맞게 클러스터 5로 설정
# 5개로 쪼개진 클러스터에 각각 smote 적용하여 데이터 다양성 높이면서 클러스터 특성 유지
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X)

# SMOTE 적용
smote = SMOTE(random_state=42)
X_resampled, clusters_resampled = smote.fit_resample(X, clusters)

# 더미 변수를 원래 값으로 변환
inverse_transformer = pipeline.named_steps['preprocessor'].named_transformers_['cat']
oversampled_df = pd.DataFrame(inverse_transformer.inverse_transform(X_resampled), columns=columns_to_keep)

# 원래 survey_df의 인덱스를 저장
original_index = survey_df.index


오버샘플링 데이터 저장 완료: ./refer/output/survey_oversampled_v2.csv


In [35]:
# oversampled_df에서 원래 oversampled_survey_df와 동일한 행을 제거
unique_oversampled_df = oversampled_df.merge(oversampled_survey_df.drop_duplicates(), on=columns_to_keep, how='left', indicator=True)
unique_oversampled_df = unique_oversampled_df[unique_oversampled_df['_merge'] == 'left_only'].drop(columns=['_merge'])

# 최종 결과 저장
unique_oversampled_df.to_csv(output_path, index=False)
print(f"오버샘플링 데이터 저장 완료: {output_path}")

오버샘플링 데이터 저장 완료: ./refer/output/survey_oversampled_v2.csv


In [39]:
oversampled_df

,설문자의 나이(만)는 어떻게 되십니까?,설문자의 해당사항을 체크해주세요.,설문자의 MBTI는 무엇입니까?,설문자의 성별은 어떻게 되십니까?,본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위],본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위],본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위],본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [4 순위],본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [5 순위]
0,25~30세,구직자,intj,남,건강,여가시간,학업 및 자기계발,업무,집안일
1,25~30세,구직자,intj,남,건강,여가시간,학업 및 자기계발,집안일,업무
2,25~30세,대학생 / 고학년(3-4학년),ESFP,여,업무,학업 및 자기계발,여가시간,건강,집안일
3,20~24세,대학생 / 고학년(3-4학년),intp,남,업무,여가시간,학업 및 자기계발,건강,집안일
4,25~30세,구직자,ISTP,남,건강,집안일,학업 및 자기계발,업무,여가시간
...,...,...,...,...,...,...,...,...,...
1084885,25~30세,구직자,INFP,남,건강,학업 및 자기계발,업무,여가시간,업무
1084886,25~30세,대학생 / 고학년(3-4학년),isfj,남,업무,학업 및 자기계발,건강,여가시간,집안일
1084887,20~24세,대학생 / 저학년 (1-2학년),Estp,여,여가시간,학업 및 자기계발,업무,건강,집안일
1084888,25~30세,구직자,INFP,남,건강,학업 및 자기계발,업무,여가시간,업무


In [49]:
# isin 결과를 얻습니다
isin_result = oversampled_survey_df.isin(oversampled_df)

# 각 행의 모든 컬럼이 True인 행을 선택합니다
all_true_rows = isin_result.all(axis=1)

# True인 행만 필터링합니다
true_rows_df = oversampled_survey_df[all_true_rows]


In [56]:
a = oversampled_survey_df.isin(oversampled_df)

In [59]:
# 모든 행에 대해 isin을 적용하여 True/False 결과를 얻습니다
isin_result = oversampled_df.apply(lambda row: row.isin(oversampled_survey_df.to_dict(orient='list')).all(), axis=1)

# True인 행들을 제외하고 unique_oversampled_df를 얻습니다
unique_oversampled_df_1 = oversampled_df[~isin_result]


KeyboardInterrupt: 

In [60]:
# 모든 열을 기준으로 df1과 df를 비교하여 df1에서 df를 제거
df_diff = pd.merge(oversampled_df, oversampled_survey_df, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

MemoryError: Unable to allocate 105. GiB for an array with shape (14114693089,) and data type int64

In [51]:
unique_oversampled_df_fix = oversampled_df.loc[~oversampled_df.index.isin(true_rows_df.index)]

<class 'pandas.core.frame.DataFrame'>
Index: 1084835 entries, 55 to 1084889
Data columns (total 9 columns):
 #   Column                                                             Non-Null Count    Dtype 
---  ------                                                             --------------    ----- 
 0   설문자의 나이(만)는 어떻게 되십니까?                                              1084835 non-null  object
 1   설문자의 해당사항을 체크해주세요.                                                 1084835 non-null  object
 2   설문자의 MBTI는 무엇입니까?                                                  1084835 non-null  object
 3   설문자의 성별은 어떻게 되십니까?                                                 1084835 non-null  object
 4   본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위]  1084835 non-null  object
 5   본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위]  1084835 non-null  object
 6   본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위]  1084835 non-null  object
 7   본인이 생각하는 일과 별 

#### 전처리

In [24]:
import pandas as pd
import numpy as np


def survey_processing(file_path, output_path):
    # CSV 파일을 로드
    survey_df = pd.read_csv(file_path)
    
    # 우선순위에 대한 가중치 매핑을 정의
    priority_weights = {
        "건강": "health",
        "여가시간": "free_time",
        "학업 및 자기계발": "edu",
        "업무": "work",
        "집안일": "chores",
        None: "category_else"
    }
    
    # 각 우선순위 컬럼에 대해 가중치를 적용
    for i in range(1, 6):
        column_name = f"본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [{i} 순위]"
        survey_df[column_name] = survey_df[column_name].map(priority_weights)
    
    # 필요한 컬럼들만 선택하고 리네임
    columns_to_keep = [
        "설문자의 나이(만)는 어떻게 되십니까?",
        "설문자의 해당사항을 체크해주세요.",
        "설문자의 MBTI는 무엇입니까?",
        "설문자의 성별은 어떻게 되십니까?",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [4 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [5 순위]"
    ]
    
    survey_df = survey_df[columns_to_keep]
    
    # 컬럼명을 변환
    column_mapping = {
        "설문자의 나이(만)는 어떻게 되십니까?": "age",
        "설문자의 해당사항을 체크해주세요.": "job",
        "설문자의 MBTI는 무엇입니까?": "mbti",
        "설문자의 성별은 어떻게 되십니까?": "gender",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위]": "priority_1",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위]": "priority_2",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위]": "priority_3",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [4 순위]": "priority_4",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [5 순위]": "priority_5"
    }
    
    survey_df.rename(columns=column_mapping, inplace=True)
    
    # 새로운 컬럼 'category_else'를 추가하고 모든 값을 0으로 설정
    survey_df["category_else"] = 0
    
    # 나이, 직업, MBTI 컬럼을 숫자 코드로 변환
    age_mapping = {
        "0~14세": (0, 14),
        "15~19세": (15, 19),
        "20~24세": (20, 24),
        "25~30세": (25, 30),
        "31세 이상": (31, 45)
    }
    job_mapping = {
        "초/중학생": "000",
        "고등학생": "001",
        "대학생 / 저학년 (1-2학년)": "002",
        "대학생 / 고학년(3-4학년)": "003",
        "구직자": "004",
        "직장인": "005",
        "자영업자": "006",
        "프리랜서": "007",
        "주부": "008",
        "기타": "009"
    }
    mbti_mapping = {
        "INTJ": "00",
        "INTP": "01",
        "ENTJ": "02",
        "ENTP": "03",
        "INFJ": "04",
        "INFP": "05",
        "ENFJ": "06",
        "ENFP": "07",
        "ISTJ": "08",
        "ISFJ": "09",
        "ESTJ": "10",
        "ESFJ": "11",
        "ISTP": "12",
        "ISFP": "13",
        "ESTP": "14",
        "ESFP": "15"
    }
    gender_mapping = {
        "남": 0,
        "여": 1
    }

    # 나이, 직업, MBTI, 성별 컬럼을 숫자 코드로 변환
    survey_df['age'] = survey_df['age'].map(lambda x: np.random.randint(age_mapping[x][0], age_mapping[x][1] + 1))
    survey_df['job'] = survey_df['job'].map(job_mapping)
    survey_df['mbti'] = survey_df['mbti'].str.upper().map(mbti_mapping)
    survey_df['gender'] = survey_df['gender'].map(gender_mapping)

    # 우선순위별로 각 항목에 가중치를 적용한 값을 할당
    work_col = []
    edu_col = []
    free_time_col = []
    health_col = []
    chores_col = []

    for _, row in survey_df.iterrows():
        work = 0
        edu = 0
        free_time = 0
        health = 0
        chores = 0

        for i in range(1, 6):
            if row[f'priority_{i}'] == 'work':
                work = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'edu':
                edu = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'free_time':
                free_time = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'health':
                health = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'chores':
                chores = 1 - (i - 1) * 0.25
        
        work_col.append(work)
        edu_col.append(edu)
        free_time_col.append(free_time)
        health_col.append(health)
        chores_col.append(chores)
    
    survey_df['work'] = work_col
    survey_df['edu'] = edu_col
    survey_df['free_time'] = free_time_col
    survey_df['health'] = health_col
    survey_df['chores'] = chores_col
    
    # 사용하지 않는 우선순위 컬럼 삭제
    survey_df.drop(columns=['priority_1', 'priority_2', 'priority_3', 'priority_4', 'priority_5'], inplace=True)
    
    # 컬럼 순서 재정렬
    survey_df = survey_df[['age', 'job', 'mbti', 'gender', 'work', 'edu', 'free_time', 'health', 'chores', 'category_else']]
    
    # 수정된 데이터프레임을 새로운 CSV 파일로 저장
    survey_df.to_csv(output_path, index=False)

# 오버샘플링된 데이터를 전처리
survey_processing(f'{file_path}survey_oversampled_v2.csv', preprocessed_output_path)
print(f"전처리된 데이터가 저장되었습니다: {preprocessed_output_path}")


전처리된 데이터가 저장되었습니다: ./refer/output/survey_preprocessed_v2.csv


In [25]:
survey_processing(f'{file_path}survey.csv', f'{file_path}survey_processed.csv')

In [30]:
survey_pro = pd.read_csv(f'{file_path}survey_preprocessed_v2.csv')

In [27]:
survey_basic = pd.read_csv(f'{file_path}survey_processed.csv')

In [28]:
survey_basic.describe()

,age,job,mbti,gender,work,edu,free_time,health,chores,category_else
count,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.0
mean,29.381818,4.327273,7.381818,0.600000,0.631818,0.440909,0.500000,0.645455,0.186364,0.0
std,6.352284,1.564118,4.572425,0.494413,0.349783,0.329715,0.333333,0.342549,0.255578,0.0
min,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,25.000000,3.000000,4.000000,0.000000,0.375000,0.250000,0.250000,0.250000,0.000000,0.0
50%,28.000000,4.000000,7.000000,1.000000,0.750000,0.500000,0.500000,0.750000,0.000000,0.0
75%,33.500000,5.000000,12.000000,1.000000,1.000000,0.750000,0.750000,1.000000,0.250000,0.0
max,44.000000,8.000000,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.750000,0.0


In [31]:
survey_pro.describe()

,age,job,mbti,gender,work,edu,free_time,health,chores,category_else
count,1.084890e+06,1.084890e+06,1.084890e+06,1.084890e+06,1.084890e+06,1.084890e+06,1.084890e+06,1.084890e+06,1.084890e+06,1084890.0
mean,2.842276e+01,4.287457e+00,7.397241e+00,5.453788e-01,5.945515e-01,4.500477e-01,5.055029e-01,6.768728e-01,1.728809e-01,0.0
std,6.661775e+00,1.526338e+00,4.583843e+00,4.979367e-01,3.437593e-01,3.270557e-01,3.298051e-01,3.414033e-01,2.333875e-01,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,2.500000e+01,3.000000e+00,4.000000e+00,0.000000e+00,2.500000e-01,2.500000e-01,2.500000e-01,2.500000e-01,0.000000e+00,0.0
50%,2.800000e+01,4.000000e+00,7.000000e+00,1.000000e+00,7.500000e-01,5.000000e-01,5.000000e-01,7.500000e-01,0.000000e+00,0.0
75%,3.000000e+01,5.000000e+00,1.200000e+01,1.000000e+00,1.000000e+00,7.500000e-01,7.500000e-01,1.000000e+00,2.500000e-01,0.0
max,4.500000e+01,8.000000e+00,1.500000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,7.500000e-01,0.0


In [18]:
import pandas as pd

# CSV 파일을 불러옵니다.
pre_path = f'{file_path}survey_preprocessed_v2.csv'
data = pd.read_csv(pre_path)

# 동일한 행을 찾기 위해 필요한 열을 지정합니다.
specified_columns = ['job', 'mbti', 'gender', 'work', 'edu', 'free_time', 'health', 'chores', 'category_else']

# 지정된 열에 대해 중복된 행을 찾습니다.
identical_rows = data.duplicated(subset=specified_columns, keep=False)

# 중복된 행만 추출합니다.
duplicate_data = data[identical_rows]


In [19]:
duplicate_data

,age,job,mbti,gender,work,edu,free_time,health,chores,category_else
0,30,4,0,0,0.25,0.50,0.75,1.00,0.00,0
1,27,4,0,0,0.00,0.50,0.75,1.00,0.25,0
2,30,3,15,1,1.00,0.75,0.50,0.25,0.00,0
3,23,3,1,0,1.00,0.50,0.75,0.25,0.00,0
4,28,4,12,0,0.25,0.50,0.00,1.00,0.75,0
...,...,...,...,...,...,...,...,...,...,...
1084885,29,4,5,0,0.00,0.75,0.25,1.00,0.00,0
1084886,30,3,9,0,1.00,0.75,0.25,0.50,0.00,0
1084887,21,2,14,1,0.50,0.75,1.00,0.25,0.00,0
1084888,25,4,5,0,0.00,0.75,0.25,1.00,0.00,0


In [ ]:

# 중복된 행 중에서 고유한 조합을 찾습니다.
unique_combinations = duplicate_data[specified_columns].drop_duplicates()

# 고유한 조합을 기준으로 50개의 행을 무작위로 선택합니다.
sampled_data = unique_combinations.sample(n=50, random_state=1)

# 결과를 출력합니다.
print(sampled_data)


## 전처리기 (fin)

In [69]:
import pandas as pd
import numpy as np

preprocessed_output_path = f'{file_path}survey_pre_v1.csv'


def survey_processing(file_path, output_path):
    # CSV 파일을 로드
    survey_df = pd.read_csv(file_path)
    
    # 우선순위에 대한 가중치 매핑을 정의
    priority_weights = {
        "건강": "health",
        "여가시간": "free_time",
        "학업 및 자기계발": "edu",
        "업무": "work",
        "집안일": "chores",
        None: "category_else"
    }
    
    # 각 우선순위 컬럼에 대해 가중치를 적용
    for i in range(1, 6):
        column_name = f"본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [{i} 순위]"
        survey_df[column_name] = survey_df[column_name].map(priority_weights)
    
    # 필요한 컬럼들만 선택하고 리네임
    columns_to_keep = [
        "설문자의 나이(만)는 어떻게 되십니까?",
        "설문자의 해당사항을 체크해주세요.",
        "설문자의 MBTI는 무엇입니까?",
        "설문자의 성별은 어떻게 되십니까?",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [4 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [5 순위]"
    ]
    
    survey_df = survey_df[columns_to_keep]
    
    # 컬럼명을 변환
    column_mapping = {
        "설문자의 나이(만)는 어떻게 되십니까?": "age",
        "설문자의 해당사항을 체크해주세요.": "job",
        "설문자의 MBTI는 무엇입니까?": "mbti",
        "설문자의 성별은 어떻게 되십니까?": "gender",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위]": "priority_1",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위]": "priority_2",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위]": "priority_3",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [4 순위]": "priority_4",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [5 순위]": "priority_5"
    }
    
    survey_df.rename(columns=column_mapping, inplace=True)
    
    # 새로운 컬럼 'category_else'를 추가하고 모든 값을 0으로 설정
    survey_df["category_else"] = 0
    
    # 나이, 직업, MBTI 컬럼을 숫자 코드로 변환
    age_mapping = {
        "0~14세": (0, 14),
        "15~19세": (15, 19),
        "20~24세": (20, 24),
        "25~30세": (25, 30),
        "31세 이상": (31, 45)
    }
    job_mapping = {
        "초/중학생": "000",
        "고등학생": "001",
        "대학생 / 저학년 (1-2학년)": "002",
        "대학생 / 고학년(3-4학년)": "003",
        "구직자": "004",
        "직장인": "005",
        "자영업자": "006",
        "프리랜서": "007",
        "주부": "008",
        "기타": "009"
    }
    mbti_mapping = {
        "INTJ": "00",
        "INTP": "01",
        "ENTJ": "02",
        "ENTP": "03",
        "INFJ": "04",
        "INFP": "05",
        "ENFJ": "06",
        "ENFP": "07",
        "ISTJ": "08",
        "ISFJ": "09",
        "ESTJ": "10",
        "ESFJ": "11",
        "ISTP": "12",
        "ISFP": "13",
        "ESTP": "14",
        "ESFP": "15"
    }
    gender_mapping = {
        "남": 0,
        "여": 1
    }

    # 나이, 직업, MBTI, 성별 컬럼을 숫자 코드로 변환
    survey_df['age'] = survey_df['age'].map(lambda x: np.random.randint(age_mapping[x][0], age_mapping[x][1] + 1))
    survey_df['job'] = survey_df['job'].map(job_mapping)
    survey_df['mbti'] = survey_df['mbti'].str.upper().map(mbti_mapping)
    survey_df['gender'] = survey_df['gender'].map(gender_mapping)

    # 우선순위별로 각 항목에 가중치를 적용한 값을 할당
    work_col = []
    edu_col = []
    free_time_col = []
    health_col = []
    chores_col = []

    for _, row in survey_df.iterrows():
        work = 0
        edu = 0
        free_time = 0
        health = 0
        chores = 0

        for i in range(1, 6):
            if row[f'priority_{i}'] == 'work':
                work = i
            elif row[f'priority_{i}'] == 'edu':
                edu = i
            elif row[f'priority_{i}'] == 'free_time':
                free_time = i
            elif row[f'priority_{i}'] == 'health':
                health = i
            elif row[f'priority_{i}'] == 'chores':
                chores = i
        
        work_col.append(work)
        edu_col.append(edu)
        free_time_col.append(free_time)
        health_col.append(health)
        chores_col.append(chores)
    
    survey_df['work'] = work_col
    survey_df['edu'] = edu_col
    survey_df['free_time'] = free_time_col
    survey_df['health'] = health_col
    survey_df['chores'] = chores_col
    
    # 사용하지 않는 우선순위 컬럼 삭제
    survey_df.drop(columns=['priority_1', 'priority_2', 'priority_3', 'priority_4', 'priority_5'], inplace=True)
    
    # 컬럼 순서 재정렬
    survey_df = survey_df[['age', 'job', 'mbti', 'gender', 'work', 'edu', 'free_time', 'health', 'chores', 'category_else']]
    
    # 수정된 데이터프레임을 새로운 CSV 파일로 저장
    survey_df.to_csv(output_path, index=False)

# 오버샘플링된 데이터를 전처리
survey_processing(f'{file_path}survey.csv', preprocessed_output_path)
print(f"전처리된 데이터가 저장되었습니다: {preprocessed_output_path}")


전처리된 데이터가 저장되었습니다: ./refer/output/survey_pre_v1.csv


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# CSV 파일 경로
file_path = './refer/output/'
data_path = f'{file_path}survey.csv'
output_path = f'{file_path}survey_oversampled_v2.csv'
preprocessed_output_path = f'{file_path}survey_preprocessed_v2.csv'

# CSV 파일 불러오기
survey_df = pd.read_csv(data_path)

# 필요한 컬럼만 선택
columns_to_keep = [
    "설문자의 나이(만)는 어떻게 되십니까?",
    "설문자의 해당사항을 체크해주세요.", # 직업
    "설문자의 MBTI는 무엇입니까?",
    "설문자의 성별은 어떻게 되십니까?",
    "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위]",
    "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위]",
    "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위]",
    "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [4 순위]",
    "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [5 순위]"
]
survey_df = survey_df[columns_to_keep]

# 오버샘플링을 수행하는 함수
def oversample_data(df, target_size=700000):
    current_size = len(df)
    if (target_size - current_size) > current_size:
        oversample_count = target_size - current_size
        oversampled_df = resample(df, replace=True, n_samples=oversample_count, random_state=42)
        return pd.concat([df, oversampled_df], axis=0)
    else:
        return resample(df, replace=True, n_samples=target_size, random_state=42)

# 데이터를 70만 개로 오버샘플링
oversampled_survey_df = oversample_data(survey_df, target_size=600000)

# 더미 변수 생성
categorical_features = columns_to_keep

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# 전처리 파이프라인
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# 데이터 변환
X = pipeline.fit_transform(oversampled_survey_df)

# KMeans 클러스터링
# 5순위 특성값에 맞게 클러스터 5로 설정
# 5개로 쪼개진 클러스터에 각각 smote 적용하여 데이터 다양성 높이면서 클러스터 특성 유지
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X)

# SMOTE 적용
smote = SMOTE(random_state=42)
X_resampled, clusters_resampled = smote.fit_resample(X, clusters)

# 더미 변수를 원래 값으로 변환
inverse_transformer = pipeline.named_steps['preprocessor'].named_transformers_['cat']
oversampled_df = pd.DataFrame(inverse_transformer.inverse_transform(X_resampled), columns=columns_to_keep)

# 원래 survey_df의 인덱스를 저장
original_index = survey_df.index


In [ ]:
# oversampled_df에서 원래 oversampled_survey_df와 동일한 행을 제거


# 최종 결과 저장
unique_oversampled_df.to_csv(output_path, index=False)
print(f"오버샘플링 데이터 저장 완료: {output_path}")

In [30]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# CSV 파일 경로
file_path = './refer/output/'
data_path = f'{file_path}survey_pre_v1.csv'
output_path = f'{file_path}survey_fin_v1.csv'

# CSV 파일 불러오기
survey_df = pd.read_csv(data_path)

# 오버샘플링을 수행하는 함수
def oversample_data(df, target_size=700000):
    current_size = len(df)
    if (target_size - current_size) > current_size:
        oversample_count = target_size - current_size
        oversampled_df = resample(df, replace=True, n_samples=oversample_count, random_state=42)
        return pd.concat([df, oversampled_df], axis=0)
    else:
        return resample(df, replace=True, n_samples=target_size, random_state=42)

# 데이터를 70만 개로 오버샘플링
oversampled_survey_df = oversample_data(survey_df, target_size=60)

# 수치형 데이터 표준화
numeric_features = ["work", "edu", "free_time", "health", "chores", "category_else"]

preprocessor = Pipeline(steps=[('scaler', StandardScaler())])

# 데이터 변환
X = preprocessor.fit_transform(oversampled_survey_df[numeric_features])

# KMeans 클러스터링
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X)

# SMOTE 적용
smote = SMOTE(random_state=42)
X_resampled, clusters_resampled = smote.fit_resample(X, clusters)

# Resampled 데이터를 원래 데이터프레임 형식으로 변환
oversampled_df = pd.DataFrame(X_resampled, columns=numeric_features)



Resulting number of rows: 85


In [31]:
oversampled_df

,work,edu,free_time,health,chores,category_else,age,job,mbti,gender
0,1.083228,-1.651811,-0.808450,0.568796,0.720823,0.0,37,8,11,1
1,-0.314485,-0.115243,1.744549,-0.982467,0.159143,0.0,23,3,1,0
2,0.384371,-0.883527,-1.659449,1.344428,0.720823,0.0,23,3,3,1
3,1.782084,-0.115243,-0.808450,-0.982467,-2.087579,0.0,25,5,7,1
4,-1.013342,0.653042,1.744549,0.568796,-0.964218,0.0,40,8,11,1
...,...,...,...,...,...,...,...,...,...,...
80,-1.013342,0.653042,1.744549,0.568796,-0.964218,0.0,41,8,5,1
81,-0.819059,1.207742,1.130129,0.137543,-1.276514,0.0,30,4,12,0
82,-1.013342,-0.741900,0.841258,2.072399,-2.018552,0.0,40,8,11,1
83,-1.013342,0.651845,1.744549,0.568796,-0.963343,0.0,25,4,1,0


In [ ]:

# 나머지 컬럼들도 오버샘플링된 길이에 맞춰 할당
other_columns = [col for col in oversampled_survey_df.columns if col not in numeric_features]
for col in other_columns:
    # SMOTE로 생성된 샘플 수만큼 임의로 데이터 할당
    repeated_values = pd.Series(oversampled_survey_df[col].values).sample(n=len(X_resampled), replace=True, random_state=42).values
    oversampled_df[col] = repeated_values

# 결과 저장
oversampled_df.to_csv(output_path, index=False)
print(f"Resulting number of rows: {oversampled_df.shape[0]}")


In [11]:
test_df = final_df.iloc[:, 0:10]

In [14]:
test_df.equals(survey_df)

False

In [15]:
cols = test_df.columns

In [ ]:
test_df[cols] == survey_df[cols]

In [17]:
data = {
    'A' : [1, 2, 3], 
    'B' : [4, 5, 6],
    'C' : [7, 8, 9] 
}
df1 = pd.DataFrame(data)
df2 = pd.DataFrame(data)

In [18]:
df1 == df2

,A,B,C
0,True,True,True
1,True,True,True
2,True,True,True


In [22]:
b = test_df.isin(survey_df)

In [27]:
b.any(axis=1).value_counts()

False    45
True     40
Name: count, dtype: int64

In [29]:
b.any(axis=0)

work             False
edu              False
free_time        False
health           False
chores           False
category_else    False
age               True
job               True
mbti              True
gender            True
dtype: bool

In [21]:
import pandas as pd
from sklearn.utils import resample
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# CSV 파일 경로
file_path = './refer/output/'
data_path = f'{file_path}survey_pre_v1.csv'
output_path = f'{file_path}smote_v1.csv'

# CSV 파일 불러오기
survey_df = pd.read_csv(data_path)

# 오버샘플링을 수행하는 함수
def oversample_data(df, target_size=700000):
    current_size = len(df)
    if (target_size - current_size) > current_size:
        oversample_count = target_size - current_size
        oversampled_df = resample(df, replace=True, n_samples=oversample_count, random_state=42)
        return pd.concat([df, oversampled_df], axis=0)
    else:
        return resample(df, replace=True, n_samples=target_size, random_state=42)

# 데이터를 6만 개로 오버샘플링
oversampled_survey_df = oversample_data(survey_df, target_size=50000)

# 수치형 데이터 선택
numeric_features = ["work", "edu", "free_time", "health", "chores"]

# KMeans 클러스터링
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(oversampled_survey_df[numeric_features])

# SMOTE 적용
smote = SMOTE(random_state=42)
X_resampled, clusters_resampled = smote.fit_resample(oversampled_survey_df[numeric_features], clusters)

# SMOTE로 생성된 데이터에 1씩 더하기
X_resampled += 1

# Resampled 데이터를 원래 데이터프레임 형식으로 변환
oversampled_df = pd.DataFrame(X_resampled, columns=numeric_features)

# 나머지 컬럼들도 오버샘플링된 길이에 맞춰 할당
other_columns = [col for col in oversampled_survey_df.columns if col not in numeric_features]
for col in other_columns:
    # SMOTE로 생성된 샘플 수만큼 임의로 데이터 할당
    repeated_values = pd.Series(oversampled_survey_df[col].values).sample(n=len(X_resampled), replace=True, random_state=42).values
    oversampled_df[col] = repeated_values

# 결과 저장
oversampled_df.to_csv(output_path, index=False)
print(f"Resulting number of rows: {oversampled_df.shape[0]}")


Resulting number of rows: 71715


In [24]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# CSV 파일 경로
file_path = './refer/output/'
data_path = f'{file_path}survey_pre_v1.csv'
output_path = f'{file_path}smote_v2.csv'

# CSV 파일 불러오기
survey_df = pd.read_csv(data_path)

# 오버샘플링을 수행하는 함수
def oversample_data(df, target_size=30000):
    current_size = len(df)
    if (target_size - current_size) > current_size:
        oversample_count = target_size - current_size
        oversampled_df = resample(df, replace=True, n_samples=oversample_count, random_state=42)
        return pd.concat([df, oversampled_df], axis=0)
    else:
        return resample(df, replace=True, n_samples=target_size, random_state=42)

# 데이터를 3만 개로 오버샘플링
oversampled_survey_df = oversample_data(survey_df, target_size=30000)

# 수치형 데이터 선택
numeric_features = ["work", "edu", "free_time", "health", "chores"]

# KMeans 클러스터링
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(oversampled_survey_df[numeric_features])

# 각 클래스의 샘플 수를 기준으로 sampling_strategy 생성
unique, counts = np.unique(clusters, return_counts=True)
class_counts = dict(zip(unique, counts))

# 최빈 클래스의 샘플 수를 기준으로 소수 클래스 비율 차등 조정
max_count = max(class_counts.values())
sampling_strategy = {}
for cls, count in class_counts.items():
    if count < max_count:
        # 클래스의 샘플 수가 적을수록 더 많은 샘플을 생성
        ratio = max_count / count
        sampling_strategy[cls] = int(count * ratio)

# SMOTE 적용
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, clusters_resampled = smote.fit_resample(oversampled_survey_df[numeric_features], clusters)

# SMOTE로 생성된 데이터에 1씩 더하기
X_resampled += 1

# Resampled 데이터를 원래 데이터프레임 형식으로 변환
oversampled_df = pd.DataFrame(X_resampled, columns=numeric_features)

# 나머지 컬럼들도 오버샘플링된 길이에 맞춰 할당
other_columns = [col for col in oversampled_survey_df.columns if col not in numeric_features]
for col in other_columns:
    # SMOTE로 생성된 샘플 수만큼 임의로 데이터 할당
    repeated_values = pd.Series(oversampled_survey_df[col].values).sample(n=len(X_resampled), replace=True, random_state=42).values
    oversampled_df[col] = repeated_values

# 결과 저장
oversampled_df.to_csv(output_path, index=False)
print(f"Resulting number of rows: {oversampled_df.shape[0]}")


Resulting number of rows: 45030


In [25]:
oversampled_df

,work,edu,free_time,health,chores,age,job,mbti,gender,category_else
0,7,6,5,4,8,22,3,1,1,0
1,8,6,5,4,7,30,4,3,0,0
2,4,5,6,7,8,26,3,15,1,0
3,4,6,5,7,8,40,5,6,1,0
4,7,6,8,4,5,20,2,14,1,0
...,...,...,...,...,...,...,...,...,...,...
45025,5,7,4,6,8,30,3,9,0,0
45026,4,5,6,7,8,32,5,5,1,0
45027,4,6,5,7,8,25,5,12,1,0
45028,5,7,4,6,8,26,4,15,1,0


In [71]:
survey_df['work'].value_counts()

work
3    18
4    14
5     9
6     7
7     6
0     1
Name: count, dtype: int64

In [72]:
oversampled_df['work'].value_counts()

work
3    31249
4    22431
5    19624
7    17554
6    13925
0     1982
Name: count, dtype: int64

In [26]:
(oversampled_df['work'] == oversampled_df['edu']).sum()

0

In [27]:
(oversampled_df['work'] == oversampled_df['free_time']).sum()

0

In [28]:
(oversampled_df['work'] == oversampled_df['health']).sum()

0

In [29]:
(oversampled_df['work'] == oversampled_df['chores']).sum()

0

In [30]:
(oversampled_df['work'] == oversampled_df['category_else']).sum()

0

In [32]:
v1 = pd.read_csv(f'{file_path}smote_v1.csv')

In [34]:
v2 = pd.read_csv(f'{file_path}smote_v2.csv')

In [35]:
user_data = pd.concat((v1, v2))

In [ ]:
user_data

In [37]:
user_data.to_csv(f'{file_path}user_data.csv', index=False)

In [39]:
import pandas as pd

# CSV 파일 경로
data_path = f'{file_path}user_data.csv'
output_path = f'{file_path}user_data_ranked_v1.csv'

# CSV 파일 불러오기
survey_df = pd.read_csv(data_path)

# 수치형 데이터 선택
numeric_features = ["work", "edu", "free_time", "health", "chores", "category_else"]

# 각 행의 값을 순위로 변환하여 1부터 6까지 나타내기
for index, row in survey_df.iterrows():
    rank = row[numeric_features].rank(method='first', ascending=False).astype(int)
    survey_df.loc[index, numeric_features] = rank.values

# 결과 저장
survey_df.to_csv(output_path, index=False)
print(f"Resulting number of rows: {survey_df.shape[0]}")


Resulting number of rows: 116745
